In [67]:
# Modules
import os, glob
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterstats as rs
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import geoplot as gplt
import warnings
warnings.filterwarnings('ignore')

os.chdir('C:/Users/mccoo/OneDrive/mcook')

# Load the HISDAC-US BUPR grids
buprdir = os.path.join('data', 'hisdac_us', 'BUPR')
rasters = glob.glob(buprdir+"/*.tif", recursive=True)
print(rasters)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# Load in the FIRED events
path = os.path.join(os.getcwd(), 'fired', 'data', 'events', 'mod', 'fired_events_conus_to2020_qc.gpkg')
fired = gpd.read_file(path, driver='GPKG')
fired.drop('bupr_sum', axis=1)
fired.drop('bupr_sum1k', axis=1)

# Read in the spatial ICS + FIRED
path = os.path.join(os.getcwd(), 'ics209-plus-fired', 'data', 'spatial', 'mod', 'ics-fired', 'ics-fired_west_plus_2001to2020.gpkg')
ics_fired = gpd.read_file(path, driver="GPKG")
ics_fired = ics_fired.loc[~ics_fired.index.duplicated(keep='first')]
ics_fired.reset_index(inplace=True, drop=True)
print(ics_fired.info())

# Read in state boundaries, subset to western states
path = os.path.join(
    os.getcwd(), 'data', 'boundaries', 'political', 
    'TIGER', 'tl19_us_states_west_plus.gpkg')
states = gpd.read_file(path, driver="GPKG")

# Simple map
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
f, ax = plt.subplots(1)
states.plot(ax=ax,aspect=1)
ics_fired.plot(
    ax=ax,column="FINAL_ACRES",legend=True, markersize=0.2, aspect=1,
    norm=matplotlib.colors.LogNorm()) 
plt.show()
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Imports successful ...")

['data\\hisdac_us\\BUPR\\BUPR_2000.tif', 'data\\hisdac_us\\BUPR\\BUPR_2005.tif', 'data\\hisdac_us\\BUPR\\BUPR_2010.tif', 'data\\hisdac_us\\BUPR\\BUPR_2015.tif']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8408 entries, 0 to 8407
Columns: 123 entries, id to geometry
dtypes: float64(63), geometry(1), int64(4), object(55)
memory usage: 7.9+ MB
None
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


ValueError: Invalid vmin or vmax

ValueError: Invalid vmin or vmax

<Figure size 432x288 with 2 Axes>

In [ ]:
### Identify county overlap

In [57]:
### Define zonal statistics function
def get_zonal_stats(vector, raster, stats):
    # Run zonal statistics, store result in geopandas dataframe
    result = rs.zonal_stats(vector, raster, stats=stats, geojson_out=True)
    return gpd.GeoDataFrame.from_features(result) 

In [58]:
### Get the years and filter the geodataframe
# BUPR years (semidecade)
bupr_years = [int(os.path.basename(r)[5:9]) for r in rasters]
print(bupr_years)

# Fire years
fire_years = list(fired['ig_year'].unique())
for i in range(0, len(fire_years)):
    fire_years[i] = int(fire_years[i])

print("min burn year: "+str(min(fire_years)))
print("max burn year: "+str(max(fire_years)))

[2000, 2005, 2010, 2015]
min burn year: 2001
max burn year: 2020


In [59]:
### Now, perform zonal statistics for each semi-decade
sums = [] # Empty list for results    
for yr in bupr_years:
    grid = os.path.join(buprdir, 'BUPR_'+str(yr)+'.tif')
    if yr == 2015:
        polys = fired[(fired['ig_year'] >= yr) & (fired['ig_year'] <= max(fire_years))]
        print(list(polys['ig_year'].unique()))
        zs = get_zonal_stats(polys, grid, stats='sum')
        zs = zs.rename(columns={'sum': 'bupr_sum'})
        sums.append(zs)
    else:
        polys = fired[(fired['ig_year'] >= yr) & (fired['ig_year'] < yr+5)]
        print(list(polys['ig_year'].unique()))
        zs = get_zonal_stats(polys, grid, stats='sum')
        zs = zs.rename(columns={'sum': 'bupr_sum'})
        sums.append(zs)
print(len(sums))

[2001.0, 2002.0, 2003.0, 2004.0]
[2005.0, 2006.0, 2007.0, 2008.0, 2009.0]
[2010.0, 2011.0, 2012.0, 2013.0, 2014.0]
[2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0]
4


In [60]:
### Merge results into one dataframe
print("Merging data frames ...")
gdf = gpd.GeoDataFrame(pd.concat(sums), crs=sums[0].crs)
print(gdf.info())

Merging data frames ...
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 95648 entries, 0 to 29335
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geometry    95648 non-null  geometry
 1   STUSPS      95648 non-null  object  
 2   bupr_sum    95648 non-null  float64 
 3   bupr_sum1k  95648 non-null  float64 
 4   eco_mode    95399 non-null  object  
 5   eco_name    95419 non-null  object  
 6   eco_type    95648 non-null  object  
 7   event_dur   95648 non-null  float64 
 8   fid_        67226 non-null  float64 
 9   fsr_km2_dy  95648 non-null  float64 
 10  fsr_px_dy   95648 non-null  float64 
 11  id          95648 non-null  float64 
 12  ig_date     95648 non-null  object  
 13  ig_day      95648 non-null  float64 
 14  ig_month    95648 non-null  float64 
 15  ig_utm_x    95648 non-null  float64 
 16  ig_utm_y    95648 non-null  float64 
 17  ig_year     95648 non-null  float64 
 18  last_date   95

In [61]:
# 1km buffer
def buffer(row):
     return row.geometry.buffer(1000)
# copy of the dataframe
buffered = fired.copy()
# apply the function
buffered['geometry'] = buffered.apply(buffer, axis=1)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 95648 entries, 0 to 95647
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   fid_        67226 non-null  float64 
 1   id          95648 non-null  float64 
 2   ig_date     95648 non-null  object  
 3   ig_day      95648 non-null  float64 
 4   ig_month    95648 non-null  float64 
 5   ig_year     95648 non-null  float64 
 6   last_date   95648 non-null  object  
 7   event_dur   95648 non-null  float64 
 8   tot_pix     95648 non-null  float64 
 9   tot_ar_km2  95648 non-null  float64 
 10  fsr_px_dy   95648 non-null  float64 
 11  fsr_km2_dy  95648 non-null  float64 
 12  mx_grw_px   95648 non-null  float64 
 13  mn_grw_px   95648 non-null  float64 
 14  mu_grw_px   95648 non-null  float64 
 15  mx_grw_km2  95648 non-null  float64 
 16  mn_grw_km2  95648 non-null  float64 
 17  mu_grw_km2  95648 non-null  float64 
 18  mx_grw_dte  95648 non-null  object  
 

In [62]:
### Buffer zonal stats
sums1km = [] #Empty list for results    
for yr in bupr_years:
    grid = os.path.join(buprdir, 'BUPR_'+str(yr)+'.tif')
    if yr == 2015:
        polys = buffered[(buffered['ig_year'] >= yr) & (buffered['ig_year'] <= max(fire_years))]
        print(list(polys['ig_year'].unique()))
        zs = get_zonal_stats(polys, grid, stats='sum')
        zs = zs.rename(columns={'sum': 'bupr_sum1km'})
        sums1km.append(zs)
    else:
        polys = buffered[(buffered['ig_year'] >= yr) & (buffered['ig_year'] < yr+5)]
        print(list(polys['ig_year'].unique()))
        zs = get_zonal_stats(polys, grid, stats='sum')
        zs = zs.rename(columns={'sum': 'bupr_sum1km'})
        sums1km.append(zs)
print(len(sums1km))

[2001.0, 2002.0, 2003.0, 2004.0]
[2005.0, 2006.0, 2007.0, 2008.0, 2009.0]
[2010.0, 2011.0, 2012.0, 2013.0, 2014.0]
[2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0]
4


In [63]:
# 4km buffer
def buffer(row):
     return row.geometry.buffer(4000)
# copy of the dataframe
buffered = fired.copy()
# apply the function
buffered['geometry'] = buffered.apply(buffer, axis=1)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 95648 entries, 0 to 95647
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   fid_        67226 non-null  float64 
 1   id          95648 non-null  float64 
 2   ig_date     95648 non-null  object  
 3   ig_day      95648 non-null  float64 
 4   ig_month    95648 non-null  float64 
 5   ig_year     95648 non-null  float64 
 6   last_date   95648 non-null  object  
 7   event_dur   95648 non-null  float64 
 8   tot_pix     95648 non-null  float64 
 9   tot_ar_km2  95648 non-null  float64 
 10  fsr_px_dy   95648 non-null  float64 
 11  fsr_km2_dy  95648 non-null  float64 
 12  mx_grw_px   95648 non-null  float64 
 13  mn_grw_px   95648 non-null  float64 
 14  mu_grw_px   95648 non-null  float64 
 15  mx_grw_km2  95648 non-null  float64 
 16  mn_grw_km2  95648 non-null  float64 
 17  mu_grw_km2  95648 non-null  float64 
 18  mx_grw_dte  95648 non-null  object  
 

In [64]:
### Buffer zonal stats
sums4km = [] #Empty list for results    
for yr in bupr_years:
    grid = os.path.join(buprdir, 'BUPR_'+str(yr)+'.tif')
    if yr == 2015:
        polys = buffered[(buffered['ig_year'] >= yr) & (buffered['ig_year'] <= max(fire_years))]
        print(list(polys['ig_year'].unique()))
        zs = get_zonal_stats(polys, grid, stats='sum')
        zs = zs.rename(columns={'sum': 'bupr_sum4km'})
        sums4km.append(zs)
    else:
        polys = buffered[(buffered['ig_year'] >= yr) & (buffered['ig_year'] < yr+5)]
        print(list(polys['ig_year'].unique()))
        zs = get_zonal_stats(polys, grid, stats='sum')
        zs = zs.rename(columns={'sum': 'bupr_sum4km'})
        sums4km.append(zs)
print(len(sums4km))

[2001.0, 2002.0, 2003.0, 2004.0]
[2005.0, 2006.0, 2007.0, 2008.0, 2009.0]
[2010.0, 2011.0, 2012.0, 2013.0, 2014.0]
[2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0]
4


In [65]:
### Merge results into one dataframe
print("Merging data frames ...")
gdf1km = gpd.GeoDataFrame(pd.concat(sums1km), crs=sums1km[0].crs)[['id', 'bupr_sum1km']]
gdf4km = gpd.GeoDataFrame(pd.concat(sums4km), crs=sums4km[0].crs)[['id', 'bupr_sum4km']]
# Merge attributes
df1 = gdf.merge(gdf1km, on='id')
df2 = df1.merge(gdf4km, on='id')[['id', 'bupr_sum', 'bupr_sum1km', 'bupr_sum4km']]
print(df2)
# Write to file
print("Writing output file ...")
out_file = os.path.join('fired', 'data', 'events', 'mod', 'fired_conus_bupr_sum.csv')
df2.to_csv(out_file)

Merging data frames ...
             id  bupr_sum  bupr_sum  bupr_sum1km  bupr_sum4km
0         286.0       0.0       0.0          1.0        847.0
1         287.0       0.0       0.0          0.0          0.0
2         288.0       0.0       0.0          0.0          0.0
3         289.0       0.0       0.0          0.0          0.0
4         290.0       0.0       0.0          0.0          0.0
...         ...       ...       ...          ...          ...
95643  137848.0       1.0       1.0          4.0        192.0
95644  137862.0       0.0       0.0          6.0        576.0
95645  135453.0    1498.0    1498.0       5463.0      35968.0
95646  122850.0     303.0     303.0       1311.0       6614.0
95647  117191.0     110.0     110.0        565.0       4545.0

[95648 rows x 5 columns]
Writing output file ...
